## Goal: Latent representations of news outlets 

In [161]:
from surprise import accuracy, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise.reader import Reader
from surprise import Dataset
import pandas as pd
import numpy as np

In [2]:
IN_PATH = "/shared/3/projects/newsDiffusion/data/processed/fullDataWithClustNums.tsv"

#we are bringing in the data with demographic information and cluster ids 
#hence, merged
#MVP path merged = pd.read_csv("/shared/3/projects/benlitterer/localNews/data/interim/SingleNE_85_clustered.tsv", sep="\t")
merged = pd.read_csv(IN_PATH, sep="\t")
merged = merged.dropna(subset=["clustNum"])



<ipython-input-2-8555fc74f582>:6: DtypeWarning: Columns (2,3,10,11,13,14,17,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  merged = pd.read_csv(IN_PATH, sep="\t")


In [3]:
merged["clustNum"] = merged["clustNum"].astype(int)

In [144]:
outletGroups = merged[["source", "clustNum"]].groupby("source").agg(set)
outletGroups["len"] = outletGroups["clustNum"].apply(len)
outletGroups = outletGroups.sort_values("len").head(400)
outletDict = outletGroups.to_dict(orient="index")

In [96]:
#MAX_CLUSTNUM = max(merged["clustNum"])
#for TESTING: 
MAX_CLUSTNUM = 50000
OUTLET_COUNT = len(outletGroups)

In [83]:
#choiceMat = np.zeros((OUTLET_COUNT, MAX_CLUSTNUM), dtype=int)

In [97]:
from tqdm import tqdm

In [102]:
#TODO: check indexing

#change the appropriate indices to have zeros 
#this is the list telling us which index is which outlet in our matrix 
sortedOutlets = sorted(list(outletGroups.index))

#get the data in long format
longDict = {"outletID":[], "storyID":[], "covered":[]}
for outlet in tqdm(sortedOutlets): 
    
    #the indices corresponding to stories that are covered for this outlet 
    artIndices = outletDict[outlet]["clustNum"]
    
    #add the outlet ID and story ID
    #if we have a max clust num of 5, that means we have 6 clusters (0 indexed)
    longDict["outletID"] += [outlet for i in range(0, MAX_CLUSTNUM+1)]
    longDict["storyID"] += [i for i in range(0, MAX_CLUSTNUM+1)]
    
    coverageVec = list(np.zeros(MAX_CLUSTNUM+1))
    for storyIndex in outletDict[outlet]["clustNum"]: 
        if storyIndex <= MAX_CLUSTNUM: 
            coverageVec[storyIndex] = 1
    longDict["covered"] += coverageVec

100%|██████████| 200/200 [00:00<00:00, 291.56it/s]


In [105]:
coverageDf = pd.DataFrame(longDict)
len(coverageDf)

2000200

In [106]:
reader = Reader(rating_scale=(0, 1))
dataset = Dataset.load_from_df(coverageDf, reader)

In [ ]:
# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(dataset, test_size=0.25)

# We'll use the famous SVD algorithm.
algo = SVD(n_factors=300)


In [131]:
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
testOutput = algo.test(testset)


In [132]:
predictions = [output.est for output in testOutput]
binaryPreds = [int(round(pred, 0)) for pred in predictions]
gts = [output.r_ui for output in testOutput]

In [133]:
len(predictions)

500050

In [134]:
from sklearn.metrics import f1_score

In [135]:
f1_score(gts, binaryPreds)

0.016349977905435263

In [142]:
len(trainset.ur[3])

7522